# Kaggle Comptetition: Black Friday Sale Prediction

__Task__: The objective to predict the primary product category given other features of the product. You may also create your own features.<br>

__Metrics__: The evaluation metric for this competition is Accuracy.<br>

__Other metrics (optional)__While you are working on the problem, you should also check the precision and recall of your models. However, this is
for your learning, and will not be considered in the evaluation.<br>

__Submission Format__<br>
The solution file will be a CSV file consisting of Product_ID and your predicted class. It should contain two columns: Product_ID and Product_Category_1.<br>

_import neccessary library_

In [383]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

_Importing the data_<br>
Courtersy to Analytics Vidhya

In [384]:
df = pd.read_csv('black_friday_data_kaggle.csv')

_Exploring the data_:<br>
The data provided consists of users and the product IDs of the products they purchased. <br>
There are some other features as well. For some of the products, you will see that the target (Product_Category_1) is marked as -1. Those are your test data, i.e.the Product IDs for which you need to predict the category.<br>
Note that you will need to transform the data to a one-row per product form for this.



In [385]:
df = pd.get_dummies(data=df, columns=['Gender', 'Age', 'Occupation',
                                   'City_Category', 'Marital_Status',
                                   'Stay_In_Current_City_Years',])

df = df.drop(['Product_Category_2', 'Product_Category_3'], axis = 1)

df = df.dropna()


In [358]:
print(list(df))
df.head()

['Unnamed: 0', 'User_ID', 'Product_ID', 'Product_Category_1', 'Purchase', 'Gender_F', 'Gender_M', 'Age_0-17', 'Age_18-25', 'Age_26-35', 'Age_36-45', 'Age_46-50', 'Age_51-55', 'Age_55+', 'Occupation_0', 'Occupation_1', 'Occupation_2', 'Occupation_3', 'Occupation_4', 'Occupation_5', 'Occupation_6', 'Occupation_7', 'Occupation_8', 'Occupation_9', 'Occupation_10', 'Occupation_11', 'Occupation_12', 'Occupation_13', 'Occupation_14', 'Occupation_15', 'Occupation_16', 'Occupation_17', 'Occupation_18', 'Occupation_19', 'Occupation_20', 'City_Category_A', 'City_Category_B', 'City_Category_C', 'Marital_Status_0', 'Marital_Status_1', 'Stay_In_Current_City_Years_0', 'Stay_In_Current_City_Years_1', 'Stay_In_Current_City_Years_2', 'Stay_In_Current_City_Years_3', 'Stay_In_Current_City_Years_4+']


,Unnamed: 0,User_ID,Product_ID,Product_Category_1,Purchase,Gender_F,Gender_M,Age_0-17,Age_18-25,Age_26-35,...,City_Category_A,City_Category_B,City_Category_C,Marital_Status_0,Marital_Status_1,Stay_In_Current_City_Years_0,Stay_In_Current_City_Years_1,Stay_In_Current_City_Years_2,Stay_In_Current_City_Years_3,Stay_In_Current_City_Years_4+
0,0,1000001,P00069042,-1,8370,1,0,1,0,0,...,1,0,0,1,0,0,0,1,0,0
1,1,1000001,P00248942,1,15200,1,0,1,0,0,...,1,0,0,1,0,0,0,1,0,0
2,2,1000001,P00087842,-1,1422,1,0,1,0,0,...,1,0,0,1,0,0,0,1,0,0
3,3,1000001,P00085442,-1,1057,1,0,1,0,0,...,1,0,0,1,0,0,0,1,0,0
4,4,1000002,P00285442,8,7969,0,1,0,0,0,...,0,0,1,1,0,0,0,0,0,1


In [363]:
# df = df.groupby(['Product_ID', 'Product_Category_1'])[['Gender_F', 'Gender_M', 
#                                                   'Age_0-17', 'Age_18-25', 'Age_26-35','Age_36-45',
#                                                   'Age_46-50', 'Age_51-55', 'Age_55+',
#                                                   'Occupation_0', 'Occupation_1', 'Occupation_2',
#                                                   'Occupation_3', 'Occupation_4', 'Occupation_5',
#                                                   'Occupation_6', 'Occupation_7', 'Occupation_8',
#                                                   'Occupation_9', 'Occupation_10', 'Occupation_11',
#                                                   'Occupation_12', 'Occupation_13', 'Occupation_14',
#                                                   'Occupation_15', 'Occupation_16', 'Occupation_17',
#                                                   'Occupation_18', 'Occupation_19', 'Occupation_20',
#                                                   'City_Category_A', 'City_Category_B', 'City_Category_C', 
#                                                   'Marital_Status_0', 'Marital_Status_1',
#                                                   'Stay_In_Current_City_Years_0', 'Stay_In_Current_City_Years_1',
#                                                   'Stay_In_Current_City_Years_2', 'Stay_In_Current_City_Years_3',
#                                                   'Stay_In_Current_City_Years_4+']].mean()


df = df.groupby(['Product_ID', 'Product_Category_1'])[['Gender_F', 'Gender_M', 
                                                  'Age_0-17', 'Age_18-25', 'Age_26-35','Age_36-45',
                                                  'Age_46-50', 'Age_51-55', 'Age_55+',
                                                  'City_Category_A', 'City_Category_B', 'City_Category_C', 
                                                  'Marital_Status_0', 'Marital_Status_1',
                                                  'Stay_In_Current_City_Years_0', 'Stay_In_Current_City_Years_1',
                                                  'Stay_In_Current_City_Years_2', 'Stay_In_Current_City_Years_3',
                                                  'Stay_In_Current_City_Years_4+']].mean()


df.reset_index(inplace=True)

train = df.loc[df.Product_Category_1 != -1,:]
x_train = train.loc[:,~train.columns.isin(['Product_ID', 'Product_Category_1'])]
y_train = train.Product_Category_1 

test = df.loc[df.Product_Category_1 == -1,:]
x_test = test.loc[:,~test.columns.isin(['Product_ID', 'Product_Category_1'])]
y_test = test.Product_Category_1


clf_rf = RandomForestClassifier(n_estimators=15, criterion='entropy', random_state=0)
clf_rf.fit(X=x_train, y=y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=15, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [364]:
# clf_rf = RandomForestClassifier(n_estimators=15, criterion='entropy', random_state=0)
# clf_rf.fit(X=x_train, y=y_train)
# clf_rf.score(x_train, y_train)
pred = clf_rf.predict(x_test)
result = pd.DataFrame({'Product_ID' : test.Product_ID, 'Product_Category_1' : pred})
result = result[['Product_ID', 'Product_Category_1']]
result.to_csv('Prediction.csv', index=False)
# result.shape

In [369]:
test = pd.read_csv('BlackFriday.csv')
test = pd.get_dummies(data=test, columns=['Gender', 'Age', 'Occupation',
                                   'City_Category', 'Marital_Status',
                                   'Stay_In_Current_City_Years',])
test = test.drop(['Product_Category_2', 'Product_Category_3'], axis = 1)
test = test.dropna()

# test = test.groupby(['Product_ID', 'Product_Category_1'])[['Gender_F', 'Gender_M', 
#                                                   'Age_0-17', 'Age_18-25', 'Age_26-35','Age_36-45',
#                                                   'Age_46-50', 'Age_51-55', 'Age_55+',
#                                                   'Occupation_0', 'Occupation_1', 'Occupation_2',
#                                                   'Occupation_3', 'Occupation_4', 'Occupation_5',
#                                                   'Occupation_6', 'Occupation_7', 'Occupation_8',
#                                                   'Occupation_9', 'Occupation_10', 'Occupation_11',
#                                                   'Occupation_12', 'Occupation_13', 'Occupation_14',
#                                                   'Occupation_15', 'Occupation_16', 'Occupation_17',
#                                                   'Occupation_18', 'Occupation_19', 'Occupation_20',
#                                                   'City_Category_A', 'City_Category_B', 'City_Category_C', 
#                                                   'Marital_Status_0', 'Marital_Status_1',
#                                                   'Stay_In_Current_City_Years_0', 'Stay_In_Current_City_Years_1',
#                                                   'Stay_In_Current_City_Years_2', 'Stay_In_Current_City_Years_3',
#                                                   'Stay_In_Current_City_Years_4+']].mean()

test = test.groupby(['Product_ID', 'Product_Category_1'])[['Gender_F', 'Gender_M', 
                                                  'Age_0-17', 'Age_18-25', 'Age_26-35','Age_36-45',
                                                  'Age_46-50', 'Age_51-55', 'Age_55+',
                                                  'City_Category_A', 'City_Category_B', 'City_Category_C', 
                                                  'Marital_Status_0', 'Marital_Status_1',
                                                  'Stay_In_Current_City_Years_0', 'Stay_In_Current_City_Years_1',
                                                  'Stay_In_Current_City_Years_2', 'Stay_In_Current_City_Years_3',
                                                  'Stay_In_Current_City_Years_4+']].mean()


test.reset_index(inplace=True)

x_test = test.loc[:,~train.columns.isin(['Product_ID', 'Product_Category_1'])]
y_test = test.Product_Category_1 
# clf_rf.fit(X=x_test, y=y_test)
print(clf_rf.score(x_test, y_test))
pred = clf_rf.predict(x_test)
print(accuracy_score(pred, y_test))

0.8009936516698868
0.8009936516698868


In [370]:
df.shape

(537577, 13)

In [371]:
test.shape

(3623, 21)

In [373]:
df_product_id = set(df.Product_ID)
original_product_id = set(test.Product_ID)

In [382]:
# df_product_id  'P00343242'
# original_product_id 'P00343242'

check_overlap = [id in original_product_id for id in df_product_id]
check_overlap

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,